In [31]:
###load packages
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from http import cookiejar
from time import sleep
import random
from lxml import html
import re
import requests_cache

requests_cache.install_cache('bookstore')

The goal of this book is to scrapt data from ucdavis online book store.   
We need to get the textbooks used in  each course.

In [32]:
### create a txt to add cookie info
session = requests.session()
session.cookies = cookiejar.LWPCookieJar(filename='cookies.txt')
try:
    session.cookies.load(ignore_discard=True)
except FileNotFoundError:
    print("load cookies failed")

In [54]:
#### according to termid to get the full list of department
def getdept():
    url = 'http://ucdavisstores.com/SelectTermDept/Department'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36',
           'X-Requested-With': 'XMLHttpRequest',
           'Host': 'ucdavisstores.com',
           'Referer': 'http://ucdavisstores.com/SelectTermDept/Courses'} 
    params = {'termId': '8247'}
    response = requests.post(url, params=params, headers=headers)
    return(response)

In [52]:
###get the dataframe of department name and its id
dept = pd.DataFrame(columns=['did',"dname"])
response = getdept()
bs = BeautifulSoup(response.json(),"lxml")
i = 0
for ce in bs.findAll("li",{"class":"ordering_hover_color ordering_enabled padding10 hover_pointer bottom_border list_display"}):
    deptid = ce.attrs["data-id"]
    deptname = ce.get_text()
    dept.loc[i]={'did':deptid,"dname":deptname}
    i = i + 1

In [53]:
dept

,did,dname
0,dpt-16884,ART
1,dpt-16885,ARTHI
2,dpt-16888,AS AM
3,dpt-16895,BL ST
4,dpt-16897,C LIT
5,dpt-16898,CH E
6,dpt-16899,CH ST
7,dpt-16900,CHEM
8,dpt-16901,CHIN
9,dpt-16902,CLASS


In [77]:
dept["did"]=dept["did"].str.strip("dpt-")
dept = dept.apply(pd.to_numeric, errors='ignore')

In [58]:
######save table
dept.to_excel("dept.xlsx")

In [96]:
#### get the full list of courses open by a certain department####
cour = pd.DataFrame(columns=['did','cid',"cname"])
########## make requests ################
url = 'http://ucdavisstores.com/SelectTermDept/Courses'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36',
           'X-Requested-With': 'XMLHttpRequest',
           'Host': 'ucdavisstores.com',
           'Referer': 'http://ucdavisstores.com/SelectTermDept/Courses'} 
i = 0
for de in dept["did"]:
    params = {'termId': '8246','deptId':str(de)}
    response = requests.post(url, params=params, headers=headers)
    cs = response.text
    course = json.loads(cs)
    bs2 =BeautifulSoup(course,"lxml")
    for ce in bs2.findAll("li",{"class":"ordering_hover_color ordering_enabled padding10 hover_pointer bottom_border list_display"}):
        courseid = ce.attrs["data-id"]
        coursename = ce.get_text()
        cour.loc[i]={'did':de,'cid':courseid,"cname":coursename}
        i = i + 1

In [ ]:
#### save cour table
cour["cid"] = cour["cid"].str.strip("cou-")
cour.to_csv("course.csv")

In [49]:
###### complete the form in order to search the textbooks and save cookies
def gettext(did,cid):
    session.cookies.clear()
    rqst_url = 'http://ucdavisstores.com/SelectTermDept/CourseList'
    data = {
        'termId':'8247',
        'deptId':str(did),
        'courseId':str(cid)}
    response = session.post(rqst_url, data=data)
    print(response)
    rqst_code = response.json()
    print(rqst_code)
    for i in session.cookies:
        print(i)
    session.cookies.save()

In [13]:
########## extract key information on the page
def create_textbook_list(bs3):

    term = bs3.find("span",{'class':"No_Material_Course_Term"}).get_text()
    coursename = bs3.find("span",{'class':"No_Material_Course_Name"}).get_text()
    coursesection = bs3.find("span",{'class':"No_Material_Course_Section"}).get_text()
    Instructor = bs3.find("span",{'class':"No_Material_Course_Instructor"}).get_text()
    courseid = bs3.find("span",{'class':"No_Material_Course_ID"}).get_text()
############################################
    requireexist = bs3.findAll("p",{'class':"red Course_With_Material_Required"})
    l1 = len(requireexist)
    if requireexist!= []:
        for i in (range(0,l1)):
            book_needed = bs3.findAll("p",{'class':"red Course_With_Material_Required"})[i].get_text()
            book_name = bs3.findAll("h2",{'class':"lead bold Book_Title bottom5"})[i].get_text()
            book_author = bs3.findAll("p",{'class':"small Book_Author"})[i].get_text()
            book_edition = bs3.findAll("p",{'class':"small Book_Edition"})[i].get_text()
            book_pubtime = bs3.findAll("p",{'class':"small Book_Published"})[i].get_text()
            book_ISBN = bs3.findAll("p",{'class':"small Book_ISBN"})[i].get_text()
            textbook.loc[textbook.shape[0]] = {'term':term,'coursename':coursename,"coursesection":coursesection,
                               'Instructor':Instructor,'courseid':courseid,
                                 'book_needed':book_needed,'book_name':book_name,'book_author':book_author,
                               'book_edition':book_edition,'book_pubtime':book_pubtime,'book_ISBN':book_ISBN}
    optionalexist = bs3.findAll("p",{'class':"green Course_With_Material_Required"},text=re.compile('Optional|Special Order Only'))
    l2 = len(optionalexist)
    if optionalexist != []:
        for i in (range(l1,l1+l2)):
            book_needed = bs3.findAll("p",{'class':"green Course_With_Material_Required"},text=re.compile('Optional|Special Order Only'))[i-l1].get_text()
            book_name = bs3.findAll("h2",{'class':"lead bold Book_Title bottom5"})[i].get_text()
            book_author = bs3.findAll("p",{'class':"small Book_Author"})[i].get_text()
            book_edition = bs3.findAll("p",{'class':"small Book_Edition"})[i].get_text()
            book_pubtime = bs3.findAll("p",{'class':"small Book_Published"})[i].get_text()
            book_ISBN = bs3.findAll("p",{'class':"small Book_ISBN"})[i].get_text()
            textbook.loc[textbook.shape[0]] = {'term':term,'coursename':coursename,"coursesection":coursesection,
                               'Instructor':Instructor,'courseid':courseid,
                                 'book_needed':book_needed,'book_name':book_name,'book_author':book_author,
                               'book_edition':book_edition,'book_pubtime':book_pubtime,'book_ISBN':book_ISBN}
        

In [18]:
textbook = pd.DataFrame(columns=['term','coursename',"coursesection",'Instructor','courseid',
                                 'book_needed','book_name','book_author','book_edition',
                                 'book_pubtime','book_ISBN'])

In [20]:
begin = input("Enter begin #:")
end = input("Enter end #:")
for i in range(int(begin), int(end)+1):
    sleep(random.randrange(1,3))
    print(i)
    gettext(str(cour.iloc[i]['did']),str(cour.iloc[i]['cid']))
    sleep(random.randrange(1,3))
    header = {
    'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1"
}

    a =requests.get("http://ucdavisstores.com/CourseMaterials",cookies=session.cookies,headers=header)
    text = a.text
    bs3 = BeautifulSoup(text,"lxml")
    create_textbook_list(bs3)

0
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=3bujud3u0fve5jbqvnknlls5 for ucdavisstores.com/>
l1= 0
l2= 0
1
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=r5mlpemyplrl54tgitsupwig for ucdavisstores.com/>
l1= 2
0
1
l2= 0
2
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=iiqp4gsdbygttgfxm5d4xipy for ucdavisstores.com/>
l1= 0
l2= 0
3
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=fdzmcal1kqnx2q42yateqlk2 for ucdavisstores.com/>
l1= 2
0
1
l2= 0
4
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=ecjxl1p1ab1w5xlxrqrilr2j for ucdavisstores.com/>
l1= 0
l2= 0
5
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=pf1uuhepj3qsle1bykrcj50t for ucdavisstores.com/>
l1= 0
l2= 0
6
<Response [200]>
{'retval': True, 'mesg': None}
<Cookie ASP.NET_SessionId=5vswsvhjhtquq2weqj0omkpz for ucdavisstores.com/>
l1= 0
l2= 0
7
<Response [200]>
{'retval': Tru

In [21]:
textbook

,term,coursename,coursesection,Instructor,courseid,book_needed,book_name,book_author,book_edition,book_pubtime,book_ISBN
0,Term: Winter 18 |,Name: AAS 016 |,Section: 001 |,"Instructor: Mukiibi, Elizabeth |",Course ID: 40181 |,Required,DEATH+THE KING'S HORSEMAN,Author: SOYINKA,Edition: 75,Published Date: 1975,ISBN: 9780393322996
1,Term: Winter 18 |,Name: AAS 016 |,Section: 001 |,"Instructor: Mukiibi, Elizabeth |",Course ID: 40181 |,Required,TRIAL OF DEDAN KIMATHI,Author: THIONG'0,Edition: 13,Published Date: 2013,ISBN: 9781478611318
2,Term: Winter 18 |,Name: AAS 050 |,Section: ALL |,"Instructor: WHITE, E |",Course ID: |,Required,BEYOND BLACKFACE,Author: BRUNDAGE,Edition: 11,Published Date: 2011,ISBN: 9780807871843
3,Term: Winter 18 |,Name: AAS 050 |,Section: ALL |,"Instructor: WHITE, E |",Course ID: |,Required,"TOMS,COONS,MULATTOES,MAMMIES,+BUCKS",Author: BOGLE,Edition: 5TH 16,Published Date: 2016,ISBN: 9780826429537


In [ ]:
textbook.to_csv("textbook("+str(begin)+"-"+str(end)+").csv")